![Rampart Logo](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/logo.png?raw=1)

Auge is an image classification model. Its main target's to recognize common photos to determine a few flat publication's features. Each image belongs to a specific realty, having a bunch of photo & panorama recognized **twinkle** can better predict apartments' order.

## I/O
Required images lie at `/content/drive/MyDrive/Scientific/v3/Images/` .

Currently the DB containes images of two types: simple photos & wide (360 deg) panoramas. Until a stable classification model panoramas should be omitted. Final classifier must be stored into `/content/drive/MyDrive/Scientific/v3/Models/auge.pth` .

## Classes
- `luxury` is a flat with rich furniture, huge rooms, chandeliers, fireplaces, etc.

![Luxury 1](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/luxury1.webp?raw=1)
![Luxury 2](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/luxury2.webp?raw=1)
![Luxury 3](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/luxury3.webp?raw=1)

- `comfort` is the most suitable for an ordinary citizen apartments. Clean, neat, sometimes minimalistic, average area, qualitive furniture, etc.

![Comfort 1](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/comfort1.webp?raw=1)
![Comfort 2](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/comfort2.webp?raw=1)
![Comfort 3](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/comfort3.webp?raw=1)

- `construction` is a flat without a finished design. No doors, floor, supplies, wallpapers, ceiling, furniture, etc. Typically, new buildings contain these apartments.

![Construction 1](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/construction1.webp?raw=1)
![Construction 2](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/construction2.webp?raw=1)
![Construction 3](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/construction3.webp?raw=1)

- `junk` is an old flat image. Probably, the whole apartments should belong to a dormitory, Khrushchevka or gostinka.

![Junk 1](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/junk1.webp?raw=1)
![Junk 2](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/junk2.webp?raw=1)
![Junk 3](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/junk3.webp?raw=1)

- `excess` is the trash category. Actually, all exterior photos, outlines & posters lie here.

![Excess 1](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/excess1.webp?raw=1)
![Excess 2](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/excess2.webp?raw=1)
![Excess 3](https://github.com/xXxRisingTidexXx/rampart/blob/improved-accuracy/images/excess3.webp?raw=1)

In [4]:
%matplotlib inline

In [13]:
from plotly.graph_objs import Pie, Figure, Scatter
from plotly.figure_factory import create_annotated_heatmap
from re import match
from numpy import arange, trace, sum
from glob import glob
from pandas import DataFrame, concat
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, ToTensor, Resize
from torch.nn import Module, Conv2d, Linear, ReLU, CrossEntropyLoss, Sequential, MaxPool2d
from torch.optim import Adam
from torch import no_grad, save, max, load, zeros, cat, long, device
from torch.cuda import is_available
from PIL.Image import open
from time import time
from google.colab.drive import mount, flush_and_unmount
from uuid import uuid4

In [6]:
mount('/content/drive')

Mounted at /content/drive


In [7]:
labels = ['luxury', 'comfort', 'junk', 'construction', 'excess']
mappings = {l: i for i, l in enumerate(labels)}

In [8]:
def parse(path):
    result = match(r'^.*/(\w+)\.(\w+)\.webp$', path)
    return ('unknown', 'unknown') if not result else result.groups()    

In [9]:
images = DataFrame({'path': glob('/content/drive/MyDrive/Scientific/V3/Images/*.webp')})
images['label'] = images['path'].apply(lambda p: mappings[parse(p)[1]])

In [10]:
counts = images['label'].value_counts().sort_index()
figure = Figure()
figure.add_trace(Pie(labels=[labels[i] for i in counts.index], values=counts.values))
figure.update_layout(margin={'t': 30, 'r': 0, 'b': 0, 'l': 0})
figure.show(renderer='colab')

In [11]:
class AugeDataset(Dataset):
    def __init__(self, data):
        self._data = data.values
        self._transforms = Compose([ToTensor(), Resize((460, 620))])

    def __len__(self):
        return len(self._data)

    def __getitem__(self, index):
        path, label = self._data[index]
        return self._transforms(open(path)), label, path

In [21]:
training_images, testing_images = train_test_split(images, test_size=0.3)
training_images, validation_images = train_test_split(training_images, test_size=0.3)
batch_size = 1
training_loader = DataLoader(
    AugeDataset(
        concat(
            [
                images[images['label'] == 0].head(2),
                images[images['label'] == 1].head(2),
                images[images['label'] == 2].head(2),
                images[images['label'] == 3].head(2),
                images[images['label'] == 4].head(2)
            ]
        )
    ),
    batch_size,
    True
)
validation_loader = DataLoader(
    AugeDataset(
        concat(
            [
                images[images['label'] == 0].iloc[2:3],
                images[images['label'] == 1].iloc[2:3],
                images[images['label'] == 2].iloc[2:3],
                images[images['label'] == 3].iloc[2:3],
                images[images['label'] == 4].iloc[2:3]
            ]
        )
    ),
    batch_size
)
testing_loader = DataLoader(
    AugeDataset(
        concat(
            [
                images[images['label'] == 0].iloc[3:5],
                images[images['label'] == 1].iloc[3:5],
                images[images['label'] == 2].iloc[3:5],
                images[images['label'] == 3].iloc[3:5],
                images[images['label'] == 4].iloc[3:5]
            ]
        )
    ), 
    batch_size
)

In [22]:
class View(Module):
    def __init__(self, *shape):
        super().__init__()
        self._shape = shape

    def forward(self, x):
        return x.view(*self._shape)

In [23]:
class Auge(Module):
    def __init__(self):
        super().__init__()
        self._sequential = Sequential(
            Conv2d(3, 5, 7, 2),
            ReLU(),
            Conv2d(5, 7, 5),
            ReLU(),
            MaxPool2d(2),
            Conv2d(7, 16, 3),
            ReLU(),
            Conv2d(16, 32, 3),
            ReLU(),
            MaxPool2d(2),
            Conv2d(32, 64, 3),
            ReLU(),
            Conv2d(64, 128, 3),
            ReLU(),
            MaxPool2d(2),
            View(-1, 104448),
            Linear(104448, 5)
        )

    def forward(self, x):
        return self._sequential(x)

In [25]:
def train(unit):
    auge = Auge().to(unit)
    criterion = CrossEntropyLoss()
    optimizer = Adam(auge.parameters())
    training_losses, validation_losses = [], []
    epoch_number = 1
    for epoch in range(1, epoch_number + 1):
        start = time()
        training_loss, validation_loss = 0.0, 0.0
        auge.train()
        for batch in training_loader:
            samples, targets = batch[0].to(unit), batch[1].to(unit)
            output = auge(samples)
            print(output)
            optimizer.zero_grad()
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.item() * samples.size(0)
        auge.eval()
        with no_grad():
            for batch in validation_loader:
                samples, targets = batch[0].to(unit), batch[1].to(unit)
                validation_loss += criterion(auge(samples), targets).item() * samples.size(0)
        end = time()
        training_loss /= len(training_loader.sampler)
        validation_loss /= len(validation_loader.sampler)
        training_losses.append(training_loss)
        validation_losses.append(validation_loss)
        print(
            f'Epoch: {epoch:2}  '
            f'Training loss: {training_loss:8.6f}  '
            f'Validation loss: {validation_loss:8.6f}  '
            f'Duration: {end - start:.0f}s'
        )
    save(auge.state_dict(), f'/content/drive/MyDrive/Scientific/V3/Models/auge.{uuid4().hex}.pth')
    save(auge.state_dict(), f'/content/drive/MyDrive/Scientific/V3/Models/auge.latest.pth')
    figure = Figure()
    figure.add_trace(Scatter(x=arange(1, epoch_number + 1), y=training_losses, name='Training'))
    figure.add_trace(Scatter(x=arange(1, epoch_number + 1), y=validation_losses, name='Validation'))
    figure.update_layout(margin={'t': 30, 'r': 80, 'b': 0, 'l': 0})
    figure.show()
    return auge

In [26]:
def use(unit):
    auge = Auge().to(unit)
    auge.load_state_dict(load('/content/drive/MyDrive/Scientific/V3/Models/auge.latest.pth'))
    return auge

In [27]:
def test(auge, unit):
    auge.eval()
    actual, expected = zeros(0, dtype=long).to(unit), zeros(0, dtype=long).to(unit)
    with no_grad():
        for batch in testing_loader:
            output = auge(batch[0].to(unit))
            print(output)
            actual = cat([actual, max(output, 1)[1].view(-1)])
            expected = cat([expected, batch[1].to(unit).view(-1)])
    matrix = confusion_matrix(expected.cpu(), actual.cpu())
    figure = create_annotated_heatmap(z=matrix, x=labels, y=labels, hoverinfo='skip')
    figure.update_xaxes(title_text='Expected')
    figure.update_yaxes(title_text='Actual', autorange='reversed')
    figure.update_layout(
        margin={'t': 100, 'r': 0, 'b': 0, 'l': 0},
        title=f'Accuracy: {trace(matrix) / sum(matrix) * 100:.2f}%'
    )
    figure.show()

In [29]:
circuit = device('cuda:0' if is_available() else 'cpu')
test(train(circuit), circuit)

tensor([[-0.0152, -0.0043,  0.0137, -0.0057, -0.0027]],
       grad_fn=<AddmmBackward>)
tensor([[-1.4430, -1.4334,  1.4606, -1.4337, -1.4333]],
       grad_fn=<AddmmBackward>)
tensor([[-1.9873, -1.9778,  0.9282, -0.4181, -1.9755]],
       grad_fn=<AddmmBackward>)
tensor([[-1.8756, -1.8670,  0.3479, -0.0250, -0.9769]],
       grad_fn=<AddmmBackward>)
tensor([[-1.8926, -1.8847,  0.1928,  0.0786, -0.5833]],
       grad_fn=<AddmmBackward>)
tensor([[-1.7613, -1.7547,  0.0531,  0.0613, -0.2137]],
       grad_fn=<AddmmBackward>)
tensor([[-1.4197, -1.1272, -0.0439,  0.0077, -0.0233]],
       grad_fn=<AddmmBackward>)
tensor([[-0.9963, -0.5918, -0.0789, -0.0306,  0.0368]],
       grad_fn=<AddmmBackward>)
tensor([[-0.7500, -0.3467, -0.0881, -0.0090,  0.0389]],
       grad_fn=<AddmmBackward>)
tensor([[-0.3713, -0.1552, -0.0667, -0.0057,  0.0203]],
       grad_fn=<AddmmBackward>)
Epoch:  1  Training loss: 2.119508  Validation loss: 1.610152  Duration: 18s


tensor([[-0.0798, -0.0439, -0.0338, -0.0008, -0.0108]])
tensor([[-0.0797, -0.0441, -0.0337, -0.0009, -0.0111]])
tensor([[-0.0745, -0.0419, -0.0325, -0.0007, -0.0114]])
tensor([[-0.0791, -0.0438, -0.0336, -0.0009, -0.0111]])
tensor([[-0.0810, -0.0447, -0.0344, -0.0008, -0.0109]])
tensor([[-0.0765, -0.0426, -0.0330, -0.0008, -0.0111]])
tensor([[-0.0810, -0.0444, -0.0341, -0.0008, -0.0106]])
tensor([[-0.0779, -0.0431, -0.0334, -0.0007, -0.0109]])
tensor([[-0.0666, -0.0394, -0.0285, -0.0021, -0.0135]])
tensor([[-0.0670, -0.0400, -0.0279, -0.0026, -0.0140]])


In [ ]:
# from cv2 import imread, IMREAD_COLOR, cvtColor, COLOR_BGR2RGB, IMREAD_GRAYSCALE, imwrite, inpaint, INPAINT_TELEA
# def d3(matrix):
#     return stack((matrix, matrix, matrix), 2)
# url = '../data/auge/images/d379f0f7ed01b7290a3a905e9daba1d1276ef7d3.comfort.webp'
# gray = imread(url, IMREAD_GRAYSCALE)
# mask = copy(gray)
# mask[341:443, 125:272] = 255
# mask[:341, :] = 0
# mask[443:, :] = 0
# mask[:, :125] = 0
# mask[:, 272:] = 0
# for x in range(125, 165):
#     mask[:382 + 126 - x, x] = 0
# for x in range(210, 272):
#     mask[:x + 130, x] = 0
# name, label = parse(url)
# iplot(
#     make_subplots(cols=2, horizontal_spacing=0.03)
#     .add_trace(Image(z=d3(gray), name=''), 1, 1)
#     .add_trace(Image(z=d3(mask), name=''), 1, 2)
#     .update_layout(title=f'{name} - {label}', margin={'t': 40, 'r': 0, 'b': 0, 'l': 0}, height=380)
#     .update_xaxes(showticklabels=False)
#     .update_yaxes(showticklabels=False)
# )
# imwrite('../data/auge/masks/olymp.jpeg', mask)
# url = '../data/auge/images/b7bb98d1f53cbc8614fad2e5926975a13904e135.construction.webp'
# bgr = imread(url, IMREAD_COLOR)
# name, label = parse(url)
# iplot(
#     make_subplots(cols=2, horizontal_spacing=0.03)
#     .add_trace(Image(z=cvtColor(bgr, COLOR_BGR2RGB), name=''), 1, 1)
#     .add_trace(
#         Image(
#             z=cvtColor(
#                 inpaint(bgr, imread('../data/auge/masks/domria.jpeg', IMREAD_GRAYSCALE), 4, INPAINT_NS),
#                 COLOR_BGR2RGB
#             ),
#             name=''
#         ),
#         1,
#         2
#     )
#     .update_layout(title=f'{name} - {label}', margin={'t': 40, 'r': 0, 'b': 0, 'l': 0}, height=380)
#     .update_xaxes(showticklabels=False)
#     .update_yaxes(showticklabels=False)
# )